<a href="https://colab.research.google.com/github/anushirahatti/mit/blob/master/covid_inpatient_cases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>